# Deep Learning Preprocessing v3 

__Author:__ Jack Friedman <br>
__Date:__ 11/10/2023 <br>
__Purpose:__ Updated and condensed frame representation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Preprocessing_v2 import *
from DataLoader import load_data

## Step 0: Load and preprocess data

In [3]:
# Load data
[games_df, players_df, plays_df, tracking_df] = load_data()

loaded games df
shape: (136, 9)
-----
loaded players df
shape: (1683, 7)
-----
loaded plays df
shape: (12486, 35)
-----
loading tracking frames...
loaded tracking frames
shape: (12187398, 17)
returning 4 frames


In [4]:
# Preprocess data
tracking_df_clean = preprocess_all_df(plays_df, games_df, players_df, tracking_df)

cleaning plays_df
final plays data shape: (6840, 289)
-----

cleaning games_df
-----

cleaning players_df
-----

cleaning tracking_df
original tracking df shape: (12187398, 17)
unique play and game id combos: (6840, 2)
filtered df shape: (8458178, 17)
number of merge errors: 0
joined plays and tracking dataframes
original tracking shape: (8458178, 17)
merged data shape: (8458178, 304)
-------
joined players and tracking dataframes
original tracking shape: (8458178, 304)
merged data shape: (8458178, 306)
-------
joined games and tracking dataframes
original tracking shape: (8458178, 306)
merged data shape: (8458178, 307)
-------
Old df shape:(8458178, 346)
New df shape:(8458178, 346)
-----



## Step 1: Filter observations by frame cutoff at handoff/run event

### 1A) Get Frame cutoffs

In [5]:
# Filter for pre-handoff events 

# Step 1: Get the frames where handoff or run occurs 
frame_cutoffs = tracking_df_clean[(tracking_df_clean['event'] == 'run') | (tracking_df_clean['event'] == 'handoff')][['gameId', 'playId', 'frameId']].drop_duplicates()

# Step 2: Handle duplicate handoffs 

# # Option A: Keep the final handoff event
# frame_cutoffs = frame_cutoffs.loc[frame_cutoffs.groupby(['gameId', 'playId'])['frameId'].idxmax()]  # keeps the max frame with a duplicate

# Option B: Drop duplicate handoff plays
frame_cutoffs = frame_cutoffs.drop_duplicates(subset=['gameId', 'playId'], keep=False)

# Step 3: Rename cutoff column
frame_cutoffs = frame_cutoffs.rename(columns = {'frameId':'frame_cutoff'})
frame_cutoffs

,gameId,playId,frame_cutoff
5,2022090800,80,6
708,2022090800,101,19
1838,2022090800,146,22
3122,2022090800,191,18
4296,2022090800,299,19
...,...,...,...
8451732,2022110700,3630,18
8453136,2022110700,3686,19
8454884,2022110700,3707,19
8456220,2022110700,3740,21


In [6]:
frame_cutoffs['frame_cutoff'].describe()

count    6794.000000
mean       17.944215
std         4.214998
min         6.000000
25%        17.000000
50%        19.000000
75%        20.000000
max        49.000000
Name: frame_cutoff, dtype: float64

### 1B) Filter tracking_df at cutoffs

In [7]:
# Step 4: Merge cutoffs with the original dataframe 
tracking_df_clean = pd.merge(tracking_df_clean, frame_cutoffs, on=['gameId', 'playId'])

# Step 5: Filter tracking data before cutoff
print("shape before filter: " + str(tracking_df_clean.shape))
tracking_df_clean = tracking_df_clean[tracking_df_clean['frameId'] <= tracking_df_clean['frame_cutoff']]
print("shape after filter: " + str(tracking_df_clean.shape))

shape before filter: (8392214, 337)
shape after filter: (2802745, 337)


## Step 2: Build tensors for single frame

### 2A) Set up df and helper functions

In [ ]:
tensor_df = pd.DataFrame(columns = ['gameId', 'playId', 'frameId', 'frame_cutoff', 'field_tensor'])

In [ ]:
def is_out_of_bounds(x, y):
    i = int(x)
    j = int(y)
    return i < 0 or i > 119 or j < 0 or j > 53

In [ ]:
# Get max s, a, height, weight for normalization
max_a = tracking_df_clean['a'].max()
max_s = tracking_df_clean['s'].max()
max_height = tracking_df_clean['heightInches'].max()   
max_weight = tracking_df_clean['weight'].max()  

In [ ]:
# NEW SCHEMA
# Channel 0: Record position of this player
# Channe; 1: Ball present in this cell?
# Channel 2: % offensive players in this cell
# Channel 3: Record defensive player ratio
# Channel 4: Net velocity vector (x component, standardized 0-1)
# Channel 5: Net velocity vector (y component, standardized 0-1)
# Channel 6: Net acceleration vector (x component, standardized 0-1)
# Channel 7: Net acceleration vector (y component, standardized 0-1)

# Other channels to consider
# Channel 8: Height (average, standardized 0-1)
# Channel 9: Height (variance, standardized 0-1)
# Channel 10: % home team in this cell
# Channel 11: % away team in this cell

# Helper function taht builds a tensor for a frame of tracking data
def build_tensor(single_frame_data, max_s = max_s, max_a = max_a, max_height = max_height, max_weight = max_weight):
    # STEP 0: Create a blank matrix
    image = np.zeros((120, 54, 22))

    # STEP 1: Record ball location before looping through players in Channel 1
    i = int(single_frame_data[single_frame_data['nflId'].isnull()]['X_std'])
    j = int(single_frame_data[single_frame_data['nflId'].isnull()]['Y_std'])
    # Make sure ball is inbounds 
    if not is_out_of_bounds(i, j):
        image[i, j, 1] = 1

    # STEP 2: Drop football from dataframe
    single_frame_data = single_frame_data.dropna(subset=['nflId'])

    # STEP 3: Populate player channels, person by person
    total_num_players_checked_on_field = 0
    num_players = np.zeros((120, 54, 4))  # Depth 0 is offense channel Depth 1 is defense, depth 2 is home, depth 3 is away
    temp_values = {}   # holds temporary values for calculating variances (0 = s, 1 = a, 2 = h, 3 = w)

    for _, row in single_frame_data.iterrows():
        # Keep track of x and y locations of player (matrix indices)
        i = int(row['X_std'])
        j = int(row['Y_std'])
        
        # Make sure player is in the frame 
        if is_out_of_bounds(i, j):
            continue

        # Update total number of players checked
        total_num_players_checked_on_field += 1

        # Channel 0: Record position of this player
        image[i, j, 0] = (image[i, j, 0] * 22 + 1) / 22

        # Keep track of people on offense and defense
        if row['isOnOffense'] == 1:
            num_players[i, j, 0] += 1  # record offensive player
        else: 
            num_players[i, j, 1] += 1  # record defensive player
        total_players_in_cell = num_players[i, j, 0] + num_players[i, j, 1]
        
        # Channel 2: % offensive players in this cell
        # Channel 3: Record defensive player ratio
        image[i, j, 2] = num_players[i, j, 0] / (num_players[i, j, 0] + num_players[i, j, 1])  # calculate % offensive players at this cell
        image[i, j, 3] = num_players[i, j, 1] / (num_players[i, j, 0] + num_players[i, j, 1])  # calculate % offensive players at this cell

        # Channel 4: Velocity (average, standardized 0-1)
        image[i, j, 4] = (image[i, j, 4] * (total_players_in_cell - 1) + row['s'] / max_s) / total_players_in_cell

        # Store continuous variables so we can calculate variances at the end
        if (i, j) not in temp_values.keys():
            temp_values[(i, j)] = {'s': [],
                                'a': [],
                                'heightInches':[],
                                'weight': [],
                                'offense_dir': [],
                                'defense_dir': [],
                                'offense_o': [],
                                'defense_o': []}
        temp_values[(i, j)]['s'] += [row['s'] / max_s]  # Channel 5: Velocity (variance, standardized 0-1)
        temp_values[(i, j)]['a'] += [row['a'] / max_a]  # Channel 7: Acceleration (variance, standardized 0-1)
        temp_values[(i, j)]['heightInches'] += [row['heightInches'] / max_height]  # Channel 9: Height (variance, standardized 0-1)
        temp_values[(i, j)]['weight'] += [row['weight'] / max_weight]  # Channel 11: Weight (variance, standardized 0-1)
        
        # Channel 6: Acceleration (average, standardized 0-1)
        image[i, j, 6] = (image[i, j, 6] * (total_players_in_cell - 1) + row['a'] / max_a) / total_players_in_cell
        

        # Store offense/defense o and dir variables for later
        if row['Dir_std'] == 0:
            dir = 360
        else:
            dir = row['Dir_std']
        if row['O_std'] == 0:
            o = 360
        else:
            o = row['O_std']

        if row['isOnOffense']:
            temp_values[(i,j)]['offense_dir'] += [dir / 360]
            temp_values[(i,j)]['offense_o'] += [o / 360]
        else:
            temp_values[(i,j)]['defense_dir'] += [dir / 360]
            temp_values[(i,j)]['defense_o'] += [o / 360]

        # Channel 16: Height (average, standardized 0-1)
        image[i, j, 16] = (image[i, j, 16] * (total_players_in_cell - 1) + row['heightInches'] / max_height) / total_players_in_cell
        
        # Channel 18: Weight (average, standardized 0-1)
        image[i, j, 18] = (image[i, j, 18] * (total_players_in_cell - 1) + row['weight'] / max_weight) / total_players_in_cell

        # Keep track of people home and away
        if row['isHomeTeam'] == 1:
            num_players[i, j, 2] += 1  # record home team player
        else: 
            num_players[i, j, 3] += 1  # record away team player
        # Update channels

        # Channel 20: % home team in this cell
        # Channel 21: % away team in this cell
        image[i, j, 20] = num_players[i, j, 2] / total_players_in_cell  # calculate % home team players at this cell
        image[i, j, 21] = num_players[i, j, 3] / total_players_in_cell  # calculate % away team players at this cell

    # Compute variances 
    for (i, j) in temp_values.keys():
        # Get sample size:
        n = len(temp_values[(i,j)]['s'])

        # Channel 5: Speed (variance, standardized 0-1)
        image[i, j, 5] = np.var(temp_values[(i,j)]['s'], ddof = 1) if n > 1 else 0

        # Channel 7: Acceleration (variance, standardized 0-1)
        image[i, j, 7] = np.var(temp_values[(i,j)]['a'], ddof = 1) if n > 1 else 0

        # Channel 8: Offense dir angle (average, standardized 0-1)
        n_o = len(temp_values[(i,j)]['offense_dir'])  # keep track of # people on offense
        image[i, j, 8] = np.mean(temp_values[(i,j)]['offense_dir']) if n_o > 0 else 0

        # Channel 9: Offense dir angle (variance, standardized 0-1)
        image[i, j, 9] = np.var(temp_values[(i,j)]['offense_dir'], ddof = 1) if n_o > 1 else 0

        # Channel 10: Defense dir angle (average, standardized 0-1)
        n_d = len(temp_values[(i,j)]['defense_dir'])  # keep track of # people on defense
        image[i, j, 10] = np.mean(temp_values[(i,j)]['defense_dir']) if n_d > 0 else 0

        # Channel 11: Defense dir angle (variance, standardized 0-1)
        image[i, j, 11] = np.var(temp_values[(i,j)]['defense_dir'], ddof = 1) if n_d > 1 else 0

        # Channel 12: Offense o angle (average, standardized 0-1)
        image[i, j, 12] = np.mean(temp_values[(i,j)]['offense_o']) if n_o > 0 else 0

        # Channel 13: Offense o angle (variance, standardized 0-1)
        image[i, j, 13] = np.var(temp_values[(i,j)]['offense_o'], ddof = 1) if n_o > 1 else 0

        # Channel 14: Defense 0 angle (average, standardized 0-1)
        image[i, j, 14] = np.mean(temp_values[(i,j)]['defense_o']) if n_d > 0 else 0

        # Channel 15: Defense o angle (variance, standardized 0-1)
        image[i, j, 15] = np.var(temp_values[(i,j)]['defense_o'], ddof = 1) if n_d > 1 else 0

        # Channel 17: Height (variance, standardized 0-1)
        image[i, j, 17] = np.var(temp_values[(i,j)]['heightInches'], ddof = 1) if n > 1 else 0
        
        # Channel 19: Weight (variance, standardized 0-1)
        image[i, j, 19] = np.var(temp_values[(i,j)]['weight'], ddof = 1) if n > 1 else 0
        
        # 22. PFF rating?
    
    # Convert matrix to a tensor
    tensor = torch.from_numpy(image)
    return tensor

### Check when 2 players are in the same matrix cell (same yard^2) in a play

In [136]:
# Single
# single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 101) & (tracking_df_clean['frameId'] == 1)]
# Double
# single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 2360) & (tracking_df_clean['frameId'] == 17)]
# Double
# single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 191) & (tracking_df_clean['frameId'] == 18)]
# Triple
# single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022091100) & (tracking_df_clean['playId'] == 546) & (tracking_df_clean['frameId'] == 16)]
# 5 in one
single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022101611) & (tracking_df_clean['playId'] == 3237) & (tracking_df_clean['frameId'] == 15)]

# STEP 0: Create a blank matrix
image = np.zeros((120, 54, 22))

# STEP 1: Record ball location before looping through players in Channel 1
i = int(single_frame_data[single_frame_data['nflId'].isnull()]['X_std'])
j = int(single_frame_data[single_frame_data['nflId'].isnull()]['Y_std'])
image[i, j, 1] = 1

# STEP 2: Drop football from dataframe
single_frame_data = single_frame_data.dropna(subset=['nflId'])

# STEP 3: Populate player channels, person by person
total_num_players_checked_on_field = 0
num_players = np.zeros((120, 54, 4))  # Depth 0 is offense channel Depth 1 is defense, depth 2 is home, depth 3 is away
temp_values = {}   # holds temporary values for calculating variances (0 = s, 1 = a, 2 = h, 3 = w)

for _, row in single_frame_data.iterrows():
    # Keep track of x and y locations of player (matrix indices)
    i = int(row['X_std'])
    j = int(row['Y_std'])
    
    # Make sure player is in the frame 
    if is_out_of_bounds(i, j):
        continue

    # Update total number of players checked
    total_num_players_checked_on_field += 1

    # Channel 0: Record position of this player
    image[i, j, 0] = (image[i, j, 0] * 22 + 1) / 22

    # Keep track of people on offense and defense
    if row['isOnOffense'] == 1:
        num_players[i, j, 0] += 1  # record offensive player
    else: 
        num_players[i, j, 1] += 1  # record defensive player
    total_players_in_cell = num_players[i, j, 0] + num_players[i, j, 1]
    
    # Channel 2: % offensive players in this cell
    # Channel 3: Record defensive player ratio
    image[i, j, 2] = num_players[i, j, 0] / (num_players[i, j, 0] + num_players[i, j, 1])  # calculate % offensive players at this cell
    image[i, j, 3] = num_players[i, j, 1] / (num_players[i, j, 0] + num_players[i, j, 1])  # calculate % offensive players at this cell

    # Channel 4: Velocity (average, standardized 0-1)
    image[i, j, 4] = (image[i, j, 4] * (total_players_in_cell - 1) + row['s'] / max_s) / total_players_in_cell

    # Store continuous variables so we can calculate variances at the end
    if (i, j) not in temp_values.keys():
        temp_values[(i, j)] = {'s': [],
                               'a': [],
                               'heightInches':[],
                               'weight': [],
                               'offense_dir': [],
                               'defense_dir': [],
                               'offense_o': [],
                               'defense_o': []}
    temp_values[(i, j)]['s'] += [row['s'] / max_s]  # Channel 5: Velocity (variance, standardized 0-1)
    temp_values[(i, j)]['a'] += [row['a'] / max_a]  # Channel 7: Acceleration (variance, standardized 0-1)
    temp_values[(i, j)]['heightInches'] += [row['heightInches'] / max_height]  # Channel 9: Height (variance, standardized 0-1)
    temp_values[(i, j)]['weight'] += [row['weight'] / max_weight]  # Channel 11: Weight (variance, standardized 0-1)
    
    # Channel 6: Acceleration (average, standardized 0-1)
    image[i, j, 6] = (image[i, j, 6] * (total_players_in_cell - 1) + row['a'] / max_a) / total_players_in_cell
    

    # Channel 8: Offense dir angle (average, standardized 0-1)
    if row['Dir_std'] == 0:
        dir = 360
    else:
        dir = row['Dir_std']
    if row['O_std'] == 0:
        o = 360
    else:
        o = row['O_std']

    if row['isOnOffense']:
        temp_values[(i,j)]['offense_dir'] += [dir / 360]
        temp_values[(i,j)]['offense_o'] += [o / 360]
    else:
        temp_values[(i,j)]['defense_dir'] += [dir / 360]
        temp_values[(i,j)]['defense_o'] += [o / 360]
    
    
# Channel 9: Offense dir angle (variance, standardized 0-1)
# Channel 10: Defense dir angle (average, standardized 0-1)
# Channel 11: Defense dir angle (variance, standardized 0-1)
# Channel 12: Offense o angle (average, standardized 0-1)
# Channel 13: Offense o angle (variance, standardized 0-1)
# Channel 14: Defense o angle (average, standardized 0-1)
# Channel 15: Defense o angle (variance, standardized 0-1)

    # Channel 16: Height (average, standardized 0-1)
    image[i, j, 16] = (image[i, j, 16] * (total_players_in_cell - 1) + row['heightInches'] / max_height) / total_players_in_cell
    
    # Channel 18: Weight (average, standardized 0-1)
    image[i, j, 18] = (image[i, j, 18] * (total_players_in_cell - 1) + row['weight'] / max_weight) / total_players_in_cell

    # Keep track of people home and away
    if row['isHomeTeam'] == 1:
        num_players[i, j, 2] += 1  # record home team player
    else: 
        num_players[i, j, 3] += 1  # record away team player
    # Update channels

    # Channel 20: % home team in this cell
    # Channel 21: % away team in this cell
    image[i, j, 20] = num_players[i, j, 2] / total_players_in_cell  # calculate % home team players at this cell
    image[i, j, 21] = num_players[i, j, 3] / total_players_in_cell  # calculate % away team players at this cell



# Compute variances 
for (i, j) in temp_values.keys():
    # Get sample size:
    n = len(temp_values[(i,j)]['s'])

    # Channel 5: Speed (variance, standardized 0-1)
    image[i, j, 5] = np.var(temp_values[(i,j)]['s'], ddof = 1) if n > 1 else 0

    # Channel 7: Acceleration (variance, standardized 0-1)
    image[i, j, 7] = np.var(temp_values[(i,j)]['a'], ddof = 1) if n > 1 else 0

    # Channel 8: Offense dir angle (average, standardized 0-1)
    image[i, j, 8] = np.mean(temp_values[(i,j)]['offense_dir'])

    # Channel 9: Offense dir angle (variance, standardized 0-1)
    n_o = len(temp_values[(i,j)]['offense_dir'])  # keep track of # people on offense
    image[i, j, 9] = np.var(temp_values[(i,j)]['offense_dir'], ddof = 1) if n_o > 1 else 0

    # Channel 10: Defense dir angle (average, standardized 0-1)
    image[i, j, 10] = np.mean(temp_values[(i,j)]['defense_dir'])

    # Channel 11: Defense dir angle (variance, standardized 0-1)
    n_d = len(temp_values[(i,j)]['defense_dir'])  # keep track of # people on defense
    image[i, j, 11] = np.var(temp_values[(i,j)]['defense_dir'], ddof = 1) if n_d > 1 else 0

    # Channel 12: Offense o angle (average, standardized 0-1)
    image[i, j, 12] = np.mean(temp_values[(i,j)]['offense_o'])

    # Channel 13: Offense o angle (variance, standardized 0-1)
    image[i, j, 13] = np.var(temp_values[(i,j)]['offense_o'], ddof = 1) if n_o > 1 else 0

    # Channel 14: Defense 0 angle (average, standardized 0-1)
    image[i, j, 14] = np.mean(temp_values[(i,j)]['defense_o'])

    # Channel 15: Defense o angle (variance, standardized 0-1)
    image[i, j, 15] = np.var(temp_values[(i,j)]['defense_o'], ddof = 1) if n_d > 1 else 0

    # Channel 17: Height (variance, standardized 0-1)
    image[i, j, 17] = np.var(temp_values[(i,j)]['heightInches'], ddof = 1) if n > 1 else 0
    
    # Channel 19: Weight (variance, standardized 0-1)
    image[i, j, 19] = np.var(temp_values[(i,j)]['weight'], ddof = 1) if n > 1 else 0
    

    
    # 22. PFF rating?

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [137]:
# (41, 23)
i = 79
j = 24
vars = {'isOnOffense': 1, 
        's': max_s, 
        'a': max_a,
        'heightInches': max_height, 
        'weight': max_weight, 
        'isHomeTeam': 1,
        'O_std': 360,
        'Dir_std': 360
        }

for var in vars.keys():
    print(var)
    vals = single_frame_data[(single_frame_data['X_std'].astype(int) == i) & (single_frame_data['Y_std'].astype(int) == j)][var].values
    for val in vals:
        print(val / vars[var])
    print("Average: " + str(np.mean(np.dot(vals,(1 / vars[var])))))
    n = len(vals)
    print("Variance: " + str(np.var(np.dot(vals,(1 / vars[var])), ddof = 1))  if n > 1 else 0)
    print('-----')

isOnOffense
0.0
1.0
0.0
1.0
0.0
Average: 0.4
Variance: 0.30000000000000004
-----
s
0.03983516316528475
0.05247252527289233
0.04532966842946196
0.013461537897234158
0.0010989010528354415
Average: 0.03043955916354173
Variance: 0.00048614293714323797
-----
a
0.03414372542009295
0.01465856274579907
0.008580622095101895
0.020915266356810867
0.0017876296031462283
Average: 0.0160171612441902
Variance: 0.00015304780940915674
-----
heightInches
0.9259259259259259
0.9876543209876543
0.8888888888888888
0.9629629629629629
0.9506172839506173
Average: 0.9432098765432098
Variance: 0.0014174668495656156
-----
weight
0.6973684210526315
0.9052631578947369
0.5368421052631579
0.8552631578947368
0.7105263157894737
Average: 0.7410526315789474
Variance: 0.021137811634349028
-----
isHomeTeam
0.0
1.0
0.0
1.0
0.0
Average: 0.4
Variance: 0.30000000000000004
-----
O_std
0.6612777777777777
0.30083333333333334
0.7002777777777778
0.34397222222222223
0.7421944444444445
Average: 0.549711111111111
Variance: 0.0441089786

In [138]:
for c in range(17):
    print("channel: " + str(c))
    print(image[i, j, c])
# Channel 0: Record position of this player
# Channel 2: % offensive players in this cell
# Channel 3: Record defensive player ratio
# Channel 4: Velocity (average, standardized 0-1)
# Channel 5: Velocity (variance, standardized 0-1)
# Channel 6: Acceleration (average, standardized 0-1)
# Channel 7: Acceleration (variance, standardized 0-1)
# Channel 8: Height (average, standardized 0-1)
# Channel 9: Height (variance, standardized 0-1)
# Channel 10: Weight (average, standardized 0-1)
# Channel 11: Weight (variance, standardized 0-1)
# Channel 12: % home team in this cell
# Channel 13: % away team in this cell




# NEW SCHEMA
# Channel 0: Record position of this player
# Channe; 1: Ball present in this cell?
# Channel 2: % offensive players in this cell
# Channel 3: Record defensive player ratio
# Channel 4: Speed (average, standardized 0-1)
# Channel 5: Speed (variance, standardized 0-1)
# Channel 6: Acceleration (average, standardized 0-1)
# Channel 7: Acceleration (variance, standardized 0-1)
# Channel 8: Offense dir angle (average, standardized 0-1)
# Channel 9: Offense dir angle (variance, standardized 0-1)
# Channel 10: Defense dir angle (average, standardized 0-1)
# Channel 11: Defense dir angle (variance, standardized 0-1)
# Channel 12: Offense o angle (average, standardized 0-1)
# Channel 13: Offense o angle (variance, standardized 0-1)
# Channel 14: Defense o angle (average, standardized 0-1)
# Channel 15: Defense o angle (variance, standardized 0-1)
# Channel 16: Height (average, standardized 0-1)
# Channel 17: Height (variance, standardized 0-1)
# Channel 18: Weight (average, standardized 0-1)
# Channel 19: Weight (variance, standardized 0-1)
# Channel 20: % home team in this cell
# Channel 21: % away team in this cell

channel: 0
0.22727272727272727
channel: 1
0.0
channel: 2
0.4
channel: 3
0.6
channel: 4
0.03043955916354173
channel: 5
0.0004861429371432381
channel: 6
0.0160171612441902
channel: 7
0.00015304780940915674
channel: 8
0.3656527777777778
channel: 9
0.002487083719135804
channel: 10
0.5075277777777777
channel: 11
0.015341370370370368
channel: 12
0.3224027777777778
channel: 13
0.0009304818672839507
channel: 14
0.7012499999999999
channel: 15
0.0016375856481481515
channel: 16
0.9432098765432098


In [134]:
temp_values[(i, j)]['offense_dir']

[0.24183333333333334, 0.9283611111111112]

In [130]:
np.var([3], ddof = 1)  if n > 1 else 0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

In [129]:
temp_values[(i,j)]['defense_dir']

[0.8209722222222222]

## Find conflicts

In [ ]:
def is_out_of_bounds(x, y):
    i = int(x)
    j = int(y)
    return i < 0 or i > 119 or j < 0 or j > 53

# Keep track of number of conflicts when 2+ players are in a cell
num_conflicts = 0
pre_cutoff_rows = tracking_df_clean[tracking_df_clean['frameId'] <= tracking_df_clean['frame_cutoff']]
for (game_id, play_id), group_df in pre_cutoff_rows.groupby(['gameId', 'playId']):
    for frame_id, frame_df in group_df.groupby(['frameId']):
        # Create a blank matrix of the field
        image = np.zeros((120, 54))

        for _, row in frame_df.iterrows():
            if pd.isnull(row['nflId']):
                continue

            # Keep track of x and y locations of player (matrix indices)
            i = int(row['X_std'])
            j = int(row['Y_std'])

            # Make sure player is in the frame 
            if is_out_of_bounds(i, j):
                continue

            image[i, j] += 1

            if image[i,j] > 1:
                num_conflicts += 1
                print("found a conflict in " + str((game_id, play_id, frame_id)) + " at " + str((i, j)))
                break

print('total conflicts: ' + str(num_conflicts))

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

found a conflict in (2022090800, 101, 14) at (48, 23)
found a conflict in (2022090800, 101, 17) at (47, 25)
found a conflict in (2022090800, 146, 17) at (60, 26)
found a conflict in (2022090800, 146, 22) at (60, 25)
found a conflict in (2022090800, 191, 16) at (74, 28)
found a conflict in (2022090800, 191, 17) at (74, 26)
found a conflict in (2022090800, 191, 18) at (75, 29)
found a conflict in (2022090800, 299, 13) at (26, 25)
found a conflict in (2022090800, 299, 14) at (26, 25)
found a conflict in (2022090800, 299, 15) at (26, 22)
found a conflict in (2022090800, 299, 16) at (26, 22)
found a conflict in (2022090800, 299, 17) at (25, 28)
found a conflict in (2022090800, 299, 18) at (25, 28)
found a conflict in (2022090800, 299, 19) at (25, 28)
found a conflict in (2022090800, 343, 15) at (24, 33)
found a conflict in (2022090800, 343, 17) at (26, 28)
found a conflict in (2022090800, 393, 19) at (50, 24)


/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

found a conflict in (2022090800, 896, 17) at (43, 20)
found a conflict in (2022090800, 933, 8) at (73, 33)
found a conflict in (2022090800, 933, 9) at (73, 33)
found a conflict in (2022090800, 933, 10) at (73, 33)
found a conflict in (2022090800, 933, 13) at (73, 32)
found a conflict in (2022090800, 933, 14) at (73, 32)
found a conflict in (2022090800, 933, 15) at (73, 32)
found a conflict in (2022090800, 933, 16) at (73, 33)
found a conflict in (2022090800, 933, 17) at (73, 33)
found a conflict in (2022090800, 933, 18) at (73, 34)
found a conflict in (2022090800, 933, 19) at (73, 34)
found a conflict in (2022090800, 933, 20) at (73, 34)
found a conflict in (2022090800, 1187, 13) at (49, 22)
found a conflict in (2022090800, 1187, 14) at (49, 22)
found a conflict in (2022090800, 1187, 15) at (49, 21)
found a conflict in (2022090800, 1254, 15) at (64, 28)
found a conflict in (2022090800, 1254, 16) at (64, 28)
found a conflict in (2022090800, 1254, 17) at (60, 29)
found a conflict in (202

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

...

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

found a conflict in (2022091102, 921, 14) at (48, 29)
found a conflict in (2022091102, 921, 15) at (48, 29)
found a conflict in (2022091102, 921, 16) at (47, 27)
found a conflict in (2022091102, 921, 17) at (48, 30)
found a conflict in (2022091102, 921, 18) at (48, 30)
found a conflict in (2022091102, 921, 19) at (48, 30)
found a conflict in (2022091102, 989, 11) at (15, 22)
found a conflict in (2022091102, 989, 15) at (16, 24)
found a conflict in (2022091102, 989, 16) at (14, 20)
found a conflict in (2022091102, 989, 17) at (14, 20)
found a conflict in (2022091102, 989, 18) at (14, 20)
found a conflict in (2022091102, 1029, 15) at (16, 30)
found a conflict in (2022091102, 1029, 16) at (16, 30)
found a conflict in (2022091102, 1050, 13) at (17, 22)
found a conflict in (2022091102, 1050, 14) at (17, 22)
found a conflict in (2022091102, 1107, 16) at (53, 23)
found a conflict in (2022091102, 1107, 17) at (57, 22)
found a conflict in (2022091102, 1107, 18) at (57, 22)
found a conflict in (

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_84845/3488530923.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

found a conflict in (2022091102, 1265, 16) at (105, 21)
found a conflict in (2022091102, 1265, 17) at (105, 26)
found a conflict in (2022091102, 1265, 18) at (105, 26)
found a conflict in (2022091102, 1286, 16) at (99, 29)
found a conflict in (2022091102, 1363, 14) at (34, 27)
found a conflict in (2022091102, 1363, 17) at (33, 27)
found a conflict in (2022091102, 1363, 18) at (33, 28)
found a conflict in (2022091102, 1363, 20) at (33, 28)
found a conflict in (2022091102, 1472, 14) at (32, 23)
found a conflict in (2022091102, 1472, 15) at (33, 30)
found a conflict in (2022091102, 1472, 16) at (33, 31)
found a conflict in (2022091102, 1472, 17) at (33, 31)
found a conflict in (2022091102, 1493, 15) at (38, 30)
found a conflict in (2022091102, 1493, 18) at (37, 31)
found a conflict in (2022091102, 1493, 19) at (37, 31)


KeyboardInterrupt: 

In [ ]:
print(num_conflicts)

NameError: name 'num_conflicts' is not defined

In [ ]:
# EXAMINE Conflict in 1 frame

 # Create a blank matrix of the field
image = np.zeros((120, 54))

# found a conflict in (2022101611, 3237, 15) at (79, 24)
frame = tracking_df_clean[(tracking_df_clean['gameId'] == 2022101611) & (tracking_df_clean['playId'] == 3237) & (tracking_df_clean['frameId'] == 15)]
for _, row in frame.iterrows():
    if pd.isnull(row['nflId']):
        continue

    # Keep track of x and y locations of player (matrix indices)
    i = int(row['X_std'])
    j = int(row['Y_std'])

    # Make sure player is in the frame 
    if is_out_of_bounds(i, j):
        continue

    image[i, j] += 1

    if image[i, j] > 4:
        print(image[i, j])

image

5.0


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Test Conflicts

In [ ]:
# Conflicts to test
# (2022090800, 101, 1) at (46, 25) --> single
# (2022090800, 191, 18) at (75, 29) --> double
# (2022090800, 2360, 17) at (68, 27) --> double
# (2022091100, 546, 16) at (41, 23) --> triple
# (2022101611, 3237, 15) at (79, 24) --> 5 in one